<a href="https://colab.research.google.com/github/Mandar117/3D-Object-Detection-and-Tracking-for-Autonomous-Vehicles/blob/main/YOLOv8_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
YOLOv8 Fine-tuning on KITTI Dataset
===================================
This script prepares the KITTI dataset and fine-tunes YOLOv8
"""
!pip install ultralytics
from ultralytics import YOLO
import os
import cv2
import yaml
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from google.colab import drive

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("klemenko/kitti-dataset")

print("Path to dataset files:", path)

In [ ]:
# Set paths
DATASET_PATH = path
LEFT_IMG_PATH = os.path.join(DATASET_PATH, 'data_object_image_2/training/image_2')
LABEL_PATH = os.path.join(DATASET_PATH, 'data_object_label_2/training/label_2')

# Create YOLO format dataset directories
YOLO_DATASET_PATH = os.path.join('/kaggle/working', 'yolo_format')
os.makedirs(os.path.join(YOLO_DATASET_PATH, 'images', 'train'), exist_ok=True)
os.makedirs(os.path.join(YOLO_DATASET_PATH, 'images', 'val'), exist_ok=True)
os.makedirs(os.path.join(YOLO_DATASET_PATH, 'labels', 'train'), exist_ok=True)
os.makedirs(os.path.join(YOLO_DATASET_PATH, 'labels', 'val'), exist_ok=True)

# Define KITTI class mapping (customize as needed)
kitti_classes = ['Car', 'Van', 'Truck', 'Pedestrian', 'Person_sitting', 'Cyclist', 'Tram', 'Misc', 'DontCare']

# Map to simplified classes
yolo_class_map = {
    'Car': 0,        # Vehicle
    'Van': 0,        # Vehicle
    'Truck': 1,      # Truck
    'Pedestrian': 2, # Pedestrian
    'Person_sitting': 2, # Pedestrian
    'Cyclist': 3,    # Cyclist
    'Tram': 4,       # Tram
    'Misc': 5,       # Misc
    'DontCare': -1   # Ignore
}

# Simplified class names for YOLOv8
simplified_classes = ['Vehicle', 'Truck', 'Pedestrian', 'Cyclist', 'Tram', 'Misc']

def convert_kitti_to_yolo():
    """Convert KITTI dataset to YOLO format"""
    print("Converting KITTI dataset to YOLO format...")

    # Check if dataset paths exist
    if not os.path.exists(LEFT_IMG_PATH) or not os.path.exists(LABEL_PATH):
        print(f"Error: Image path ({LEFT_IMG_PATH}) or label path ({LABEL_PATH}) does not exist.")
        return False

    # Get all image files
    image_files = [f for f in os.listdir(LEFT_IMG_PATH) if f.endswith('.png')]

    if len(image_files) == 0:
        print("No image files found!")
        return False

    print(f"Found {len(image_files)} images")

    # Split into train/val (80/20 split)
    train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)

    print(f"Train set: {len(train_files)} images")
    print(f"Validation set: {len(val_files)} images")

    # Process train and validation sets
    process_files(train_files, 'train')
    process_files(val_files, 'val')

    # Create dataset YAML file
    create_dataset_yaml()

    print("Conversion completed successfully!")
    return True

def process_files(files, subset):
    """Process files for the given subset (train/val)"""
    for file in tqdm(files, desc=f"Processing {subset} set"):
        # Get base filename without extension
        base_name = os.path.splitext(file)[0]

        # Copy image file
        src_img_path = os.path.join(LEFT_IMG_PATH, file)
        dst_img_path = os.path.join(YOLO_DATASET_PATH, 'images', subset, file)

        # Read and write image
        img = cv2.imread(src_img_path)
        if img is None:
            print(f"Warning: Could not read image {src_img_path}")
            continue

        cv2.imwrite(dst_img_path, img)

        # Get image dimensions for normalization
        img_height, img_width = img.shape[:2]

        # Convert label
        label_file = os.path.join(LABEL_PATH, f"{base_name}.txt")
        if not os.path.exists(label_file):
            print(f"Warning: Label file {label_file} does not exist")
            continue

        convert_annotation(label_file, subset, base_name, img_width, img_height)

def convert_annotation(label_file, subset, base_name, img_width, img_height):
    """Convert KITTI annotation to YOLO format"""
    yolo_labels = []

    with open(label_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            obj_class = parts[0]

            # Skip if class should be ignored
            if obj_class not in yolo_class_map or yolo_class_map[obj_class] == -1:
                continue

            # Extract bounding box
            try:
                x1 = float(parts[4])
                y1 = float(parts[5])
                x2 = float(parts[6])
                y2 = float(parts[7])

                # Skip invalid boxes
                if x2 <= x1 or y2 <= y1:
                    continue

                # Convert to YOLO format (normalized center, width, height)
                x_center = ((x1 + x2) / 2) / img_width
                y_center = ((y1 + y2) / 2) / img_height
                width = (x2 - x1) / img_width
                height = (y2 - y1) / img_height

                # Ensure values are within range [0, 1]
                x_center = max(0, min(1, x_center))
                y_center = max(0, min(1, y_center))
                width = max(0, min(1, width))
                height = max(0, min(1, height))

                # Get YOLO class ID
                yolo_class = yolo_class_map[obj_class]

                # Add to YOLO labels
                yolo_labels.append(f"{yolo_class} {x_center} {y_center} {width} {height}")
            except Exception as e:
                print(f"Error converting annotation: {e}")
                continue

    # Write YOLO format label file
    dst_label_path = os.path.join(YOLO_DATASET_PATH, 'labels', subset, f"{base_name}.txt")
    with open(dst_label_path, 'w') as f:
        f.write('\n'.join(yolo_labels))

def create_dataset_yaml():
    """Create YOLOv8 dataset YAML file"""
    yaml_content = {
        'path': YOLO_DATASET_PATH,
        'train': 'images/train',
        'val': 'images/val',
        'names': {i: name for i, name in enumerate(simplified_classes)}
    }

    yaml_path = os.path.join(YOLO_DATASET_PATH, 'dataset.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f, sort_keys=False)

    print(f"Created dataset.yaml at {yaml_path}")

def finetune_yolov8(epochs=10, batch_size=8, img_size=640):
    """Fine-tune YOLOv8 on the KITTI dataset"""
    # Check if dataset is prepared
    yaml_path = os.path.join(YOLO_DATASET_PATH, 'dataset.yaml')
    if not os.path.exists(yaml_path):
        print("Dataset not prepared! Converting KITTI to YOLO format...")
        if not convert_kitti_to_yolo():
            print("Failed to convert dataset. Aborting fine-tuning.")
            return None

    print(f"Starting YOLOv8 fine-tuning for {epochs} epochs...")

    # Load YOLOv8 model (nano version for faster training)
    model = YOLO('yolov8n.pt')

    # Fine-tune the model
    results = model.train(
        data=yaml_path,
        epochs=epochs,
        batch=batch_size,
        imgsz=img_size,
        patience=5,  # Early stopping patience
        save=True,   # Save checkpoints
        device='0' if torch.cuda.is_available() else 'cpu'
    )

    # Save the fine-tuned model
    model_save_path = os.path.join(DATASET_PATH, 'yolov8n_kitti.pt')
    model.export(format='onnx')  # Also export to ONNX format

    print(f"Fine-tuning completed! Model saved at {model_save_path}")

    # Validate the model
    print("Validating the fine-tuned model...")
    val_results = model.val()

    return model

def visualize_training_results(model):
    """Visualize training results and metrics"""
    # Results should be saved in runs/detect/train directory
    results_dir = list(sorted(os.listdir('runs/detect')))[-1]
    results_path = os.path.join('runs/detect', results_dir)

    # Display training metrics if available
    metrics_path = os.path.join(results_path, 'results.png')
    if os.path.exists(metrics_path):
        plt.figure(figsize=(15, 10))
        plt.imshow(plt.imread(metrics_path))
        plt.axis('off')
        plt.title('Training Metrics')
        plt.show()

    # Test the model on a sample image
    sample_image_path = os.path.join(YOLO_DATASET_PATH, 'images', 'val')
    sample_images = os.listdir(sample_image_path)

    if len(sample_images) == 0:
        print("No sample images available for testing")
        return

    # Select a random sample image
    sample_img = os.path.join(sample_image_path, np.random.choice(sample_images))

    # Run inference
    results = model.predict(sample_img, conf=0.25)[0]

    # Display detection results
    img = cv2.imread(sample_img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

    # Original image
    ax1.imshow(img)
    ax1.set_title('Original Image')
    ax1.axis('off')

    # Image with detections
    detected_img = results.plot()
    ax2.imshow(detected_img)
    ax2.set_title('Detections from Fine-tuned Model')
    ax2.axis('off')

    plt.tight_layout()
    plt.show()

    # Print detection statistics
    boxes = results.boxes.cpu().numpy()
    print(f"Detected {len(boxes)} objects")

    # Count detections by class
    class_counts = {}
    for box in boxes:
        class_id = int(box.cls)
        class_name = simplified_classes[class_id]
        if class_name in class_counts:
            class_counts[class_name] += 1
        else:
            class_counts[class_name] = 1

    print("Detection counts by class:")
    for class_name, count in class_counts.items():
        print(f"  - {class_name}: {count}")

# Main execution
if __name__ == "__main__":
    # Check necessary imports
    try:
        import torch
    except ImportError:
        print("Installing torch...")
        !pip install torch torchvision
        import torch

    print("CUDA available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("GPU:", torch.cuda.get_device_name(0))

    # Prepare dataset
    print("Step 1: Converting KITTI to YOLO format")
    convert_kitti_to_yolo()

    # Fine-tune model
    print("\nStep 2: Fine-tuning YOLOv8 on KITTI dataset")
    model = finetune_yolov8(epochs=10, batch_size=8)

    # Visualize results
    print("\nStep 3: Visualizing training results")
    if model is not None:
        visualize_training_results(model)

In [4]:
!jupyter nbconvert --ClearMetadataPreprocessor.enabled=True --inplace YOLOv8_Finetuning.ipynb


[NbConvertApp] WARNING | pattern 'YOLOv8_Finetuning.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execu